In [1]:
import sys
sys.executable
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [2]:
df_ext = pd.read_csv('external_data.csv', parse_dates=['date'])
df_ext.head()

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4
0,7149,2021-01-01 00:00:00,100810,80,1,270,1.8,272.75,272.15,96,...,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7149,2021-01-01 03:00:00,100920,110,3,300,1.7,271.25,270.95,98,...,1500.0,2.0,3.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN
2,7149,2021-01-01 06:00:00,100950,30,3,290,2.6,271.95,271.65,98,...,480.0,4.0,6.0,2000.0,6.0,3.0,3000.0,NaN,NaN,NaN
3,7149,2021-01-01 09:00:00,101100,150,2,280,1.7,272.45,272.05,97,...,1740.0,3.0,3.0,2800.0,NaN,NaN,NaN,NaN,NaN,NaN
4,7149,2021-01-01 12:00:00,101110,30,0,50,1.0,276.95,274.15,82,...,330.0,4.0,6.0,570.0,7.0,6.0,810.0,NaN,NaN,NaN


In [4]:
df_ext["date"] = pd.to_datetime(df_ext["date"])

# Add national holidays data

In [5]:
holidays_df = pd.read_csv(Path('data') / 'jours_feries_metropole.csv')
holidays_df

,date,annee,zone,nom_jour_ferie
0,2001-01-01,2001,Métropole,1er janvier
1,2001-04-16,2001,Métropole,Lundi de Pâques
2,2001-05-01,2001,Métropole,1er mai
3,2001-05-08,2001,Métropole,8 mai
4,2001-05-24,2001,Métropole,Ascension
...,...,...,...,...
281,2026-07-14,2026,Métropole,14 juillet
282,2026-08-15,2026,Métropole,Assomption
283,2026-11-01,2026,Métropole,Toussaint
284,2026-11-11,2026,Métropole,11 novembre


In [6]:
holidays_df["datetime"] = pd.to_datetime(holidays_df["date"])

In [7]:
df_ext['is_holiday'] = df_ext["date"].isin(holidays_df["datetime"]).astype(int)
df_ext

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4,is_holiday
0,7149,2021-01-01 00:00:00,100810,80,1,270,1.8,272.75,272.15,96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,7149,2021-01-01 03:00:00,100920,110,3,300,1.7,271.25,270.95,98,...,2.0,3.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7149,2021-01-01 06:00:00,100950,30,3,290,2.6,271.95,271.65,98,...,4.0,6.0,2000.0,6.0,3.0,3000.0,NaN,NaN,NaN,0
3,7149,2021-01-01 09:00:00,101100,150,2,280,1.7,272.45,272.05,97,...,3.0,3.0,2800.0,NaN,NaN,NaN,NaN,NaN,NaN,0
4,7149,2021-01-01 12:00:00,101110,30,0,50,1.0,276.95,274.15,82,...,4.0,6.0,570.0,7.0,6.0,810.0,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,7149,2020-09-30 09:00:00,101540,-30,8,230,4.4,289.95,286.85,82,...,7.0,6.0,2200.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3318,7149,2020-09-30 12:00:00,101320,-210,8,190,4.9,292.05,285.55,66,...,7.0,6.0,1900.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3319,7149,2020-09-30 15:00:00,101140,-180,7,190,4.1,291.55,286.45,72,...,7.0,6.0,2200.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3320,7149,2020-09-30 18:00:00,101020,-130,6,190,2.7,290.15,285.25,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [8]:
df_ext['is_holiday'].value_counts()

0    3311
1      11
Name: is_holiday, dtype: int64

In [9]:
df_ext = df_ext.sort_values(by="date")
df_ext

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4,is_holiday
3082,7149,2020-09-01 00:00:00,102050,-10,8,340,1.6,285.75,282.55,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3083,7149,2020-09-01 03:00:00,101990,-60,6,290,1.1,283.95,282.05,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3084,7149,2020-09-01 06:00:00,102000,10,3,360,1.8,284.25,282.85,91,...,1.0,0.0,7500.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3085,7149,2020-09-01 09:00:00,101970,0,5,360,2.9,291.25,283.35,60,...,1.0,3.0,4500.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3086,7149,2020-09-01 12:00:00,101850,-110,6,40,2.6,293.95,281.25,44,...,5.0,NaN,2160.0,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,7149,2021-10-21 00:00:00,99800,50,0,230,11.8,287.05,281.45,69,...,7.0,6.0,1350.0,NaN,NaN,NaN,NaN,NaN,NaN,0
408,7149,2021-10-21 03:00:00,100270,460,3,260,14.6,286.05,279.35,64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
409,7149,2021-10-21 06:00:00,101000,720,1,240,7.7,284.05,279.95,76,...,3.0,6.0,2400.0,NaN,NaN,NaN,NaN,NaN,NaN,0
410,7149,2021-10-21 09:00:00,101230,230,1,240,7.0,286.05,280.75,70,...,1.0,3.0,4000.0,1.0,0.0,8000.0,NaN,NaN,NaN,0


# Add Paris Covid Cases and Vaccination Rates

In [10]:
covid_df = pd.read_excel(Path('data') / 'paris_covid_cases.xlsx', usecols=["Date", "Paris"])
covid_df = covid_df.rename(columns={"Date":"date", "Paris":"paris_covid"})
covid_df["date"] = pd.to_datetime(covid_df["date"])
covid_df['paris_covid'] = covid_df['paris_covid'].str.replace(u'\xa0',"").astype(int)

In [11]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543 entries, 0 to 542
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         543 non-null    datetime64[ns]
 1   paris_covid  543 non-null    int32         
dtypes: datetime64[ns](1), int32(1)
memory usage: 6.5 KB


In [12]:
df_ext = pd.merge_asof(df_ext, covid_df, on="date")

In [13]:
vac_df = pd.read_excel(Path('data') / 'vac_data_daily.xlsx')
vac_df["date"] = pd.to_datetime(vac_df["date"])
vac_df = vac_df.rename(columns={"Paris":"paris_vac"})
vac_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       317 non-null    datetime64[ns]
 1   paris_vac  317 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 5.1+ KB


In [14]:
vac_df['paris_vac'] = vac_df['paris_vac'].str.replace(u'\xa0',"").astype(int)

In [15]:
vac_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       317 non-null    datetime64[ns]
 1   paris_vac  317 non-null    int32         
dtypes: datetime64[ns](1), int32(1)
memory usage: 3.8 KB


In [16]:
df_ext = pd.merge_asof(df_ext, vac_df, on="date").fillna(0)
df_ext 

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4,is_holiday,paris_covid,paris_vac
0,7149,2020-09-01 00:00:00,102050,-10,8,340,1.6,285.75,282.55,81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,705,0.0
1,7149,2020-09-01 03:00:00,101990,-60,6,290,1.1,283.95,282.05,88,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,705,0.0
2,7149,2020-09-01 06:00:00,102000,10,3,360,1.8,284.25,282.85,91,...,7500.0,0.0,0.0,0.0,0.0,0.0,0.0,0,705,0.0
3,7149,2020-09-01 09:00:00,101970,0,5,360,2.9,291.25,283.35,60,...,4500.0,0.0,0.0,0.0,0.0,0.0,0.0,0,705,0.0
4,7149,2020-09-01 12:00:00,101850,-110,6,40,2.6,293.95,281.25,44,...,2160.0,0.0,0.0,0.0,0.0,0.0,0.0,0,705,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,7149,2021-10-21 00:00:00,99800,50,0,230,11.8,287.05,281.45,69,...,1350.0,0.0,0.0,0.0,0.0,0.0,0.0,0,318,1969.0
3318,7149,2021-10-21 03:00:00,100270,460,3,260,14.6,286.05,279.35,64,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,318,1969.0
3319,7149,2021-10-21 06:00:00,101000,720,1,240,7.7,284.05,279.95,76,...,2400.0,0.0,0.0,0.0,0.0,0.0,0.0,0,318,1969.0
3320,7149,2021-10-21 09:00:00,101230,230,1,240,7.0,286.05,280.75,70,...,4000.0,1.0,0.0,8000.0,0.0,0.0,0.0,0,318,1969.0


In [17]:
df_ext.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3322 entries, 0 to 3321
Data columns (total 62 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   numer_sta    3322 non-null   int64         
 1   date         3322 non-null   datetime64[ns]
 2   pmer         3322 non-null   int64         
 3   tend         3322 non-null   int64         
 4   cod_tend     3322 non-null   int64         
 5   dd           3322 non-null   int64         
 6   ff           3322 non-null   float64       
 7   t            3322 non-null   float64       
 8   td           3322 non-null   float64       
 9   u            3322 non-null   int64         
 10  vv           3322 non-null   int64         
 11  ww           3322 non-null   int64         
 12  w1           3322 non-null   float64       
 13  w2           3322 non-null   float64       
 14  n            3322 non-null   float64       
 15  nbas         3322 non-null   float64       
 16  hbas  

# Adding Air Quality Data

In [18]:
air_q_df = pd.read_csv('air.csv',usecols=['date',' pm25',' pm10'],parse_dates=['date'], na_values=' ')
air_q_df = air_q_df.rename(columns={" pm25":"pm25", " pm10":"pm10"})

air_q_df = air_q_df.sort_values(by='date')

mask_air_q_df = (air_q_df['date'] >= pd.to_datetime('2020-09-01'))

air_q_df = air_q_df.loc[mask_air_q_df]

air_q_df.fillna(method='ffill', inplace=True)

air_q_df[['pm25','pm10']] = air_q_df[['pm25','pm10']].apply(pd.to_numeric)

In [19]:
df_ext = pd.merge_asof(df_ext, air_q_df, on='date')

# Adding Google Mobility Data

In [20]:
google_ext_1 = pd.read_csv('2020_FR_Region_Mobility_Report.csv', parse_dates=['date'])

google_ext_1 = google_ext_1[google_ext_1['sub_region_2'] == "Paris"]

google_ext_1.reset_index(inplace=True)

google_ext_1.drop(['index','country_region_code','country_region','sub_region_1','sub_region_2','metro_area','iso_3166_2_code','census_fips_code','place_id'], axis=1, inplace=True)
google_ext_1 = google_ext_1.rename({'retail_and_recreation_percent_change_from_baseline':'retail_change','grocery_and_pharmacy_percent_change_from_baseline':'grocery_change','parks_percent_change_from_baseline':'parks_change',
                                'transit_stations_percent_change_from_baseline':'stations_change','workplaces_percent_change_from_baseline':'workplace_change','residential_percent_change_from_baseline':'residential_change'}, axis=1)

google_ext_1 = google_ext_1[google_ext_1['date'] >= pd.to_datetime('2020/09/01')]

In [21]:
google_ext_2 = pd.read_csv('2021_FR_Region_Mobility_Report.csv', parse_dates=['date'])
google_ext_2 = google_ext_2[google_ext_2['sub_region_2'] == "Paris"]
google_ext_2.reset_index(inplace=True)

google_ext_2.drop(['index','country_region_code','country_region','sub_region_1','sub_region_2','metro_area','iso_3166_2_code','census_fips_code','place_id'], axis=1, inplace=True)
google_ext_2 = google_ext_2.rename({'retail_and_recreation_percent_change_from_baseline':'retail_change','grocery_and_pharmacy_percent_change_from_baseline':'grocery_change','parks_percent_change_from_baseline':'parks_change','transit_stations_percent_change_from_baseline':'stations_change','workplaces_percent_change_from_baseline':'workplace_change','residential_percent_change_from_baseline':'residential_change'}, axis=1)

google_ext_2

,date,retail_change,grocery_change,parks_change,stations_change,workplace_change,residential_change
0,2021-01-01,-89.0,-79.0,-61.0,-72.0,-88,25.0
1,2021-01-02,-72.0,-31.0,-58.0,-40.0,-34,8.0
2,2021-01-03,-73.0,-29.0,-59.0,-36.0,-24,4.0
3,2021-01-04,-68.0,-20.0,-58.0,-41.0,-49,18.0
4,2021-01-05,-68.0,-17.0,-57.0,-40.0,-48,19.0
...,...,...,...,...,...,...,...
319,2021-11-16,-26.0,0.0,-20.0,-6.0,-22,5.0
320,2021-11-17,-25.0,-3.0,-14.0,-10.0,-23,5.0
321,2021-11-18,-23.0,1.0,-6.0,-3.0,-21,4.0
322,2021-11-19,-22.0,-1.0,-18.0,-7.0,-26,7.0


In [22]:
google_final = pd.concat([google_ext_1, google_ext_2])
google_final

,date,retail_change,grocery_change,parks_change,stations_change,workplace_change,residential_change
199,2020-09-01,-35.0,-12.0,-17.0,-27.0,-46.0,8.0
200,2020-09-02,-33.0,-15.0,-7.0,-28.0,-44.0,7.0
201,2020-09-03,-35.0,-15.0,-8.0,-20.0,-44.0,7.0
202,2020-09-04,-32.0,-16.0,-10.0,-21.0,-44.0,5.0
203,2020-09-05,-36.0,-19.0,-21.0,-16.0,-24.0,0.0
...,...,...,...,...,...,...,...
319,2021-11-16,-26.0,0.0,-20.0,-6.0,-22.0,5.0
320,2021-11-17,-25.0,-3.0,-14.0,-10.0,-23.0,5.0
321,2021-11-18,-23.0,1.0,-6.0,-3.0,-21.0,4.0
322,2021-11-19,-22.0,-1.0,-18.0,-7.0,-26.0,7.0


In [23]:
df_ext = pd.merge_asof(df_ext, google_final, on='date')

# Adding Lockdown Stringency Index

In [24]:
lock_df = pd.read_csv('stringency.csv', parse_dates=['Day'])
lock_df = lock_df[lock_df['Code']=="FRA"]
lock_df = lock_df.rename({"Day":"date"}, axis=1)
lock_df = lock_df.drop(columns=['Entity', 'Code'])

In [25]:
df_ext = pd.merge_asof(df_ext, lock_df, on='date')

# Adding Traffic Data

In [26]:
traffic_df = pd.read_csv('comptages-routiers-permanents.csv', sep=';', error_bad_lines=False, usecols=["Date et heure de comptage","Débit horaire", "Taux d'occupation", "Etat trafic"],parse_dates=["Date et heure de comptage"],)

In [27]:
traffic_df = traffic_df.sort_values(by=["Date et heure de comptage"])

In [28]:
pd.set_option('display.max_rows', 10)
traffic_df.dropna(inplace=True)
traffic_df

,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic
514752,2020-10-01 05:00:00+02:00,116.0,0.70500,Fluide
635753,2020-10-01 05:00:00+02:00,44.0,1.25611,Fluide
635725,2020-10-01 05:00:00+02:00,123.0,0.65000,Fluide
635557,2020-10-01 05:00:00+02:00,95.0,0.32722,Fluide
635529,2020-10-01 05:00:00+02:00,65.0,0.40889,Fluide
...,...,...,...,...
2752314,2021-11-09 00:00:00+01:00,305.0,2.56556,Fluide
2714206,2021-11-09 00:00:00+01:00,102.0,1.45333,Fluide
2655854,2021-11-09 00:00:00+01:00,338.0,3.31334,Fluide
3000170,2021-11-09 00:00:00+01:00,3617.0,7.05000,Fluide


In [29]:
traffic_df["Date et heure de comptage"]= traffic_df["Date et heure de comptage"].map(lambda x: str(x)[:-6])

In [30]:
traffic_df.isna().sum()

Date et heure de comptage    0
Débit horaire                0
Taux d'occupation            0
Etat trafic                  0
dtype: int64

In [31]:
traffic_df["Date et heure de comptage"] = pd.to_datetime(traffic_df["Date et heure de comptage"], format="%Y-%m-%d %H:%M:%S")
traffic_df

,Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic
514752,2020-10-01 05:00:00,116.0,0.70500,Fluide
635753,2020-10-01 05:00:00,44.0,1.25611,Fluide
635725,2020-10-01 05:00:00,123.0,0.65000,Fluide
635557,2020-10-01 05:00:00,95.0,0.32722,Fluide
635529,2020-10-01 05:00:00,65.0,0.40889,Fluide
...,...,...,...,...
2752314,2021-11-09 00:00:00,305.0,2.56556,Fluide
2714206,2021-11-09 00:00:00,102.0,1.45333,Fluide
2655854,2021-11-09 00:00:00,338.0,3.31334,Fluide
3000170,2021-11-09 00:00:00,3617.0,7.05000,Fluide


In [32]:
traffic_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2118316 entries, 514752 to 2747218
Data columns (total 4 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   Date et heure de comptage  datetime64[ns]
 1   Débit horaire              float64       
 2   Taux d'occupation          float64       
 3   Etat trafic                object        
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 80.8+ MB


In [33]:
traffic_df = traffic_df.rename(columns={"Date et heure de comptage":"date"})
traffic_df = traffic_df.rename(columns={"Débit horaire":"flow_traffic"})
traffic_df = traffic_df.rename(columns={"Taux d'occupation":"traffic_occupation"})
traffic_df = traffic_df.rename(columns={"Etat trafic":"state_traffic"})

In [34]:
traffic_df['date']

514752    2020-10-01 05:00:00
635753    2020-10-01 05:00:00
635725    2020-10-01 05:00:00
635557    2020-10-01 05:00:00
635529    2020-10-01 05:00:00
                  ...        
2752314   2021-11-09 00:00:00
2714206   2021-11-09 00:00:00
2655854   2021-11-09 00:00:00
3000170   2021-11-09 00:00:00
2747218   2021-11-09 00:00:00
Name: date, Length: 2118316, dtype: datetime64[ns]

In [35]:
traffic_df.columns

Index(['date', 'flow_traffic', 'traffic_occupation', 'state_traffic'], dtype='object')

In [36]:
traffic_df = traffic_df.set_index('date').resample('1h').mean()

In [37]:
traffic_df.reset_index(inplace=True)

In [38]:
traffic_df.isna().sum()

date                     0
flow_traffic          5255
traffic_occupation    5255
dtype: int64

In [39]:
traffic_df

,date,flow_traffic,traffic_occupation
0,2020-10-01 05:00:00,169.420448,1.105537
1,2020-10-01 06:00:00,322.942973,1.930126
2,2020-10-01 07:00:00,700.778725,4.318268
3,2020-10-01 08:00:00,962.823887,9.429756
4,2020-10-01 09:00:00,1084.473900,15.043983
...,...,...,...
9687,2021-11-08 20:00:00,924.189032,9.978718
9688,2021-11-08 21:00:00,1022.584713,6.770383
9689,2021-11-08 22:00:00,708.316970,4.464641
9690,2021-11-08 23:00:00,674.521678,4.250265


## Adding traffic data for September

Since the above data frame starts only from October and we have to add the data for september from indivisual text files

In [40]:
sep1 = pd.read_csv("week1.txt",sep=';', error_bad_lines=False, parse_dates=['t_1h'], usecols=['t_1h','q','k'])

sep1 = sep1.sort_values(by=["t_1h"])
sep1.dropna(inplace=True)
sep1 = sep1.rename(columns={"t_1h":"date",'q':'flow_traffic','k':'traffic_occupation'})
sep1 = sep1.set_index('date').resample('1h').mean()
sep1.reset_index(inplace=True)
sep1

,date,flow_traffic,traffic_occupation
0,2020-09-01 01:00:00,365.733288,2.266770
1,2020-09-01 02:00:00,236.496651,1.427286
2,2020-09-01 03:00:00,198.968768,1.127681
3,2020-09-01 04:00:00,159.882015,0.770758
4,2020-09-01 05:00:00,155.658807,0.841345
...,...,...,...
163,2020-09-07 20:00:00,1134.050752,9.651350
164,2020-09-07 21:00:00,952.380383,6.490684
165,2020-09-07 22:00:00,700.465775,4.326012
166,2020-09-07 23:00:00,598.049455,4.105919


In [41]:
#sep1.dropna(inplace=True)
#sep1

In [42]:
sep2 = pd.read_csv("week2.txt",sep=';', error_bad_lines=False, parse_dates=['t_1h'], usecols=['t_1h','q','k'])

sep2 = sep2.sort_values(by=["t_1h"])
sep2.dropna(inplace=True)
sep2 = sep2.rename(columns={"t_1h":"date",'q':'flow_traffic','k':'traffic_occupation'})

sep2 = sep2.set_index('date').resample('1h').mean()
sep2.reset_index(inplace=True)
sep2

,date,flow_traffic,traffic_occupation
0,2020-09-08 01:00:00,380.606594,2.529493
1,2020-09-08 02:00:00,254.255480,1.611897
2,2020-09-08 03:00:00,182.882016,1.107846
3,2020-09-08 04:00:00,141.799044,0.918371
4,2020-09-08 05:00:00,168.686426,0.986752
...,...,...,...
163,2020-09-14 20:00:00,1142.761356,9.441050
164,2020-09-14 21:00:00,982.453772,6.334766
165,2020-09-14 22:00:00,735.198490,4.460771
166,2020-09-14 23:00:00,625.399515,4.184440


In [43]:
pd.set_option('display.max_rows', 10)

In [44]:
sep3 = pd.read_csv("week3.txt",sep=';', error_bad_lines=False, parse_dates=['t_1h'], usecols=['t_1h','q','k'])

sep3 = sep3.sort_values(by=["t_1h"])
sep3.dropna(inplace=True)
sep3 = sep3.rename(columns={"t_1h":"date",'q':'flow_traffic','k':'traffic_occupation'})

sep3 = sep3.set_index('date').resample('1h').mean()
sep3.reset_index(inplace=True)
sep3

,date,flow_traffic,traffic_occupation
0,2020-09-15 01:00:00,403.805096,2.451891
1,2020-09-15 02:00:00,265.805850,1.490283
2,2020-09-15 03:00:00,192.862606,1.079396
3,2020-09-15 04:00:00,150.892151,0.851152
4,2020-09-15 05:00:00,172.743728,0.908775
...,...,...,...
163,2020-09-21 20:00:00,1134.989951,9.932830
164,2020-09-21 21:00:00,987.458807,6.776535
165,2020-09-21 22:00:00,722.699434,4.462714
166,2020-09-21 23:00:00,630.984073,4.105509


In [45]:
sep4 = pd.read_csv("week4.txt",sep=';', error_bad_lines=False, parse_dates=['t_1h'], usecols=['t_1h','q','k'])

sep4 = sep4.sort_values(by=["t_1h"])
sep4.dropna(inplace=True)
sep4 = sep4.rename(columns={"t_1h":"date",'q':'flow_traffic','k':'traffic_occupation'})

sep4 = sep4.set_index('date').resample('1h').mean()
sep4.reset_index(inplace=True)
sep4

,date,flow_traffic,traffic_occupation
0,2020-09-22 01:00:00,373.922689,2.380983
1,2020-09-22 02:00:00,238.093653,1.504954
2,2020-09-22 03:00:00,170.183463,1.026118
3,2020-09-22 04:00:00,137.109384,0.779777
4,2020-09-22 05:00:00,167.117388,0.937471
...,...,...,...
163,2020-09-28 20:00:00,1073.772320,7.737270
164,2020-09-28 21:00:00,891.176999,5.716770
165,2020-09-28 22:00:00,679.006131,4.128360
166,2020-09-28 23:00:00,637.341399,3.798225


In [46]:
sep5 = pd.read_csv("week5.txt",sep=';', error_bad_lines=False, parse_dates=['t_1h'], usecols=['t_1h','q','k'])

sep5 = sep5.sort_values(by=["t_1h"])
sep5.dropna(inplace=True)
sep5 = sep5.rename(columns={"t_1h":"date",'q':'flow_traffic','k':'traffic_occupation'})

sep5 = sep5.set_index('date').resample('1h').mean()
sep5.reset_index(inplace=True)
sep5

,date,flow_traffic,traffic_occupation
0,2020-09-29 01:00:00,336.555346,2.048592
1,2020-09-29 02:00:00,207.975717,1.258750
2,2020-09-29 03:00:00,143.777793,0.915306
3,2020-09-29 04:00:00,119.816643,0.742099
4,2020-09-29 05:00:00,152.278782,0.910139
...,...,...,...
43,2020-09-30 20:00:00,1201.191304,11.850245
44,2020-09-30 21:00:00,914.252177,8.310333
45,2020-09-30 22:00:00,795.180936,5.188694
46,2020-09-30 23:00:00,731.117925,4.432054


In [47]:
sept_df = pd.concat([sep1, sep2, sep3, sep4, sep5])

In [48]:
traffic_df = pd.concat([sept_df, traffic_df])

In [49]:
df_ext = pd.merge_asof(traffic_df, df_ext, on="date")

In [50]:
df_ext.columns

Index(['date', 'flow_traffic', 'traffic_occupation', 'numer_sta', 'pmer',
       'tend', 'cod_tend', 'dd', 'ff', 't', 'td', 'u', 'vv', 'ww', 'w1', 'w2',
       'n', 'nbas', 'hbas', 'cl', 'cm', 'ch', 'pres', 'niv_bar', 'geop',
       'tend24', 'tn12', 'tn24', 'tx12', 'tx24', 'tminsol', 'sw', 'tw',
       'raf10', 'rafper', 'per', 'etat_sol', 'ht_neige', 'ssfrai', 'perssfrai',
       'rr1', 'rr3', 'rr6', 'rr12', 'rr24', 'phenspe1', 'phenspe2', 'phenspe3',
       'phenspe4', 'nnuage1', 'ctype1', 'hnuage1', 'nnuage2', 'ctype2',
       'hnuage2', 'nnuage3', 'ctype3', 'hnuage3', 'nnuage4', 'ctype4',
       'hnuage4', 'is_holiday', 'paris_covid', 'paris_vac', 'pm25', 'pm10',
       'retail_change', 'grocery_change', 'parks_change', 'stations_change',
       'workplace_change', 'residential_change', 'stringency_index'],
      dtype='object')

## Adding the multimodal data

In [51]:
multimodal_df = pd.read_csv('multimodal.csv',sep=';', error_bad_lines=False, usecols=['Date et heure de comptage', 'Nombre de véhicules'], parse_dates=['Date et heure de comptage'])
multimodal_df = multimodal_df.sort_values(by=["Date et heure de comptage"])

In [52]:
multimodal_df = multimodal_df.rename(columns={"Date et heure de comptage":"date",'Nombre de véhicules':'vehicle_number'})

In [53]:
multimodal_df["date"]= multimodal_df["date"].map(lambda x: str(x)[:-6])

In [54]:
multimodal_df["date"] = pd.to_datetime(multimodal_df["date"], format="%Y-%m-%d %H:%M:%S")

In [56]:
multimodal_df = multimodal_df.set_index('date').resample('1h').mean()

In [60]:
df_ext = pd.merge_asof(df_ext, multimodal_df, on="date")

In [61]:
df_ext

,date,flow_traffic,traffic_occupation,numer_sta,pmer,tend,cod_tend,dd,ff,t,...,pm25,pm10,retail_change,grocery_change,parks_change,stations_change,workplace_change,residential_change,stringency_index,vehicle_number
0,2020-09-01 01:00:00,365.733288,2.266770,7149,102050,-10,8,340,1.6,285.75,...,51.0,32.0,-35.0,-12.0,-17.0,-27.0,-46.0,8.0,46.76,26.636364
1,2020-09-01 02:00:00,236.496651,1.427286,7149,102050,-10,8,340,1.6,285.75,...,51.0,32.0,-35.0,-12.0,-17.0,-27.0,-46.0,8.0,46.76,13.214286
2,2020-09-01 03:00:00,198.968768,1.127681,7149,101990,-60,6,290,1.1,283.95,...,51.0,32.0,-35.0,-12.0,-17.0,-27.0,-46.0,8.0,46.76,10.545455
3,2020-09-01 04:00:00,159.882015,0.770758,7149,101990,-60,6,290,1.1,283.95,...,51.0,32.0,-35.0,-12.0,-17.0,-27.0,-46.0,8.0,46.76,8.666667
4,2020-09-01 05:00:00,155.658807,0.841345,7149,101990,-60,6,290,1.1,283.95,...,51.0,32.0,-35.0,-12.0,-17.0,-27.0,-46.0,8.0,46.76,17.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10407,2021-11-08 20:00:00,924.189032,9.978718,7149,101160,-60,8,240,4.6,286.25,...,30.0,31.0,-29.0,-7.0,-20.0,-6.0,-24.0,5.0,66.67,82.757576
10408,2021-11-08 21:00:00,1022.584713,6.770383,7149,101160,-60,8,240,4.6,286.25,...,30.0,31.0,-29.0,-7.0,-20.0,-6.0,-24.0,5.0,66.67,63.359551
10409,2021-11-08 22:00:00,708.316970,4.464641,7149,101160,-60,8,240,4.6,286.25,...,30.0,31.0,-29.0,-7.0,-20.0,-6.0,-24.0,5.0,66.67,48.963855
10410,2021-11-08 23:00:00,674.521678,4.250265,7149,101160,-60,8,240,4.6,286.25,...,30.0,31.0,-29.0,-7.0,-20.0,-6.0,-24.0,5.0,66.67,44.637681


## Adding missing values

In [62]:
from sklearn.impute import KNNImputer

In [63]:
#df_ext = pd.read_csv("external_data.csv", low_memory=False, parse_dates=['date'])
#df_ext['paris_covid'] = df_ext['paris_covid'].str.replace('\xa0','').astype(int)

In [64]:
df_ext.isna().sum()

date                     0
flow_traffic          5256
traffic_occupation    5256
numer_sta                0
pmer                     0
                      ... 
stations_change          0
workplace_change         0
residential_change       0
stringency_index         0
vehicle_number          25
Length: 74, dtype: int64

In [65]:
df_missing = df_ext[['flow_traffic', 'traffic_occupation']]

In [66]:
imputer = KNNImputer(n_neighbors=3, weights="uniform")

In [67]:
missing = imputer.fit_transform(df_missing)

In [68]:
missing = pd.DataFrame(missing, columns=['flow_traffic', 'traffic_occupation'])

In [69]:
df_ext.drop(columns=['flow_traffic', 'traffic_occupation'], inplace=True)

In [70]:
df_ext = pd.concat([df_ext, missing], axis=1)

In [73]:
df_ext

,date,numer_sta,pmer,tend,cod_tend,dd,ff,t,td,u,...,retail_change,grocery_change,parks_change,stations_change,workplace_change,residential_change,stringency_index,vehicle_number,flow_traffic,traffic_occupation
0,2020-09-01 01:00:00,7149,102050,-10,8,340,1.6,285.75,282.55,81,...,-35.0,-12.0,-17.0,-27.0,-46.0,8.0,46.76,26.636364,365.733288,2.266770
1,2020-09-01 02:00:00,7149,102050,-10,8,340,1.6,285.75,282.55,81,...,-35.0,-12.0,-17.0,-27.0,-46.0,8.0,46.76,13.214286,236.496651,1.427286
2,2020-09-01 03:00:00,7149,101990,-60,6,290,1.1,283.95,282.05,88,...,-35.0,-12.0,-17.0,-27.0,-46.0,8.0,46.76,10.545455,198.968768,1.127681
3,2020-09-01 04:00:00,7149,101990,-60,6,290,1.1,283.95,282.05,88,...,-35.0,-12.0,-17.0,-27.0,-46.0,8.0,46.76,8.666667,159.882015,0.770758
4,2020-09-01 05:00:00,7149,101990,-60,6,290,1.1,283.95,282.05,88,...,-35.0,-12.0,-17.0,-27.0,-46.0,8.0,46.76,17.500000,155.658807,0.841345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10407,2021-11-08 20:00:00,7149,101160,-60,8,240,4.6,286.25,282.65,79,...,-29.0,-7.0,-20.0,-6.0,-24.0,5.0,66.67,82.757576,924.189032,9.978718
10408,2021-11-08 21:00:00,7149,101160,-60,8,240,4.6,286.25,282.65,79,...,-29.0,-7.0,-20.0,-6.0,-24.0,5.0,66.67,63.359551,1022.584713,6.770383
10409,2021-11-08 22:00:00,7149,101160,-60,8,240,4.6,286.25,282.65,79,...,-29.0,-7.0,-20.0,-6.0,-24.0,5.0,66.67,48.963855,708.316970,4.464641
10410,2021-11-08 23:00:00,7149,101160,-60,8,240,4.6,286.25,282.65,79,...,-29.0,-7.0,-20.0,-6.0,-24.0,5.0,66.67,44.637681,674.521678,4.250265


In [74]:
df_ext.to_csv("external_data.csv", index=False)